In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
#import h5py
import os

In [ ]:
# Define dataset path
DATASET_PATH = "cat_model/dataset"

# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
# Load datasets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_PATH, "train"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_PATH, "val"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_PATH, "test"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

In [ ]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

In [ ]:
# Normalize pixel values (rescale to [0,1])
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(normalization_layer(x), training=True), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# Load MobileNetV2 without top layers (pretrained on ImageNet)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

# Freeze first 100 layers
base_model.trainable = True
for layer in base_model.layers[:100]:  
    layer.trainable = False

In [ ]:
# Build custom model
# THIS MIGHT NEED ADJUSTMENT LATER
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),  # dropout to reduce overfitting
    tf.keras.layers.Dense(1, activation="sigmoid")  # Binary classification
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,  
    restore_best_weights=True
)

In [ ]:
# Train the model
EPOCHS = 20  # tends to stop at 10
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")

# Plot training history
plt.plot(history.history["accuracy"], label="train accuracy")
plt.plot(history.history["val_accuracy"], label="val accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Save
model.save("cat_detectorX.h5")
print("Model saved as cat_detectorX.h5")

In [ ]:
model.summary()

TESTING ON IMAGES

In [ ]:
import tensorflow as tf
#import keras_preprocessing
#from keras import preprocessing
from keras_preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the saved model
# Ignore the compilation warning--don't have to bother with it ATM
model = tf.keras.models.load_model("cat_detector.h5")

In [ ]:
image_path = "5200698631_f85e6dbb9d_z.jpg"

img = image.load_img(image_path, target_size=(224, 224))  # Resize
plt.imshow(img) 
plt.axis('off')
plt.show()

img_array = image.img_to_array(img) 
img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array / 255.0  # Normalize to [0, 1]

prediction = model.predict(img_array) # this needs to be adjusted

# Debug
print(f"Prediction shape: {prediction.shape}")
print(f"Prediction value: {prediction[0][0]:.4f}")

if prediction[0][0] < 0.5:
    print("It's a cat!")
else:
    print("It's not a cat.")

In [ ]:
image_path = "20231213_062238.jpg"

img = image.load_img(image_path, target_size=(224, 224))  # Resize
plt.imshow(img) 
plt.axis('off')
plt.show()

img_array = image.img_to_array(img) 
img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array / 255.0  # Normalize to [0, 1]

prediction = model.predict(img_array)

# Debug
print(f"Prediction shape: {prediction.shape}")
print(f"Prediction value: {prediction[0][0]:.4f}")

if prediction[0][0] < 0.5:
    print("It's a cat!")
else:
    print("It's not a cat.")

In [ ]:
image_path = "20240418_123224.jpg"

img = image.load_img(image_path, target_size=(224, 224))  # Resize
plt.imshow(img) 
plt.axis('off')
plt.show()

img_array = image.img_to_array(img) 
img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array / 255.0  # Normalize to [0, 1]

prediction = model.predict(img_array)

# Debug
print(f"Prediction shape: {prediction.shape}")
print(f"Prediction value: {prediction[0][0]:.4f}")

if prediction[0][0] < 0.5:
    print("It's a cat!")
else:
    print("It's not a cat.")

In [ ]:
image_path = "20231225_141845.jpg"

img = image.load_img(image_path, target_size=(224, 224))  # Resize
plt.imshow(img) 
plt.axis('off')
plt.show()

img_array = image.img_to_array(img) 
img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array / 255.0  # Normalize to [0, 1]

prediction = model.predict(img_array)
print(prediction)

# Debug
print(f"Prediction shape: {prediction.shape}")
print(f"Prediction value: {prediction[0][0]:.4f}")

if prediction[0][0] < 0.5:
    print("It's a cat!")
else:
    print("It's not a cat.")